#Définition et Calcul des KPI

**Objectif** : Définir et calculer au moins 6 indicateurs de pilotage métier pertinents pour optimiser le réseau de services publics au Togo.


## 1. Import des Librairies et Chargement des Données Nettoyées

In [1]:
# Import des librairies nécessaires
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Chargement du dataset nettoyé
data_path = '../data/cleaned_data/dataset_nettoye.csv'
df = pd.read_csv(data_path, encoding='utf-8')

print(f"✅ Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"\nColonnes disponibles : {list(df.columns)}")

# Création d'une base SQLite en mémoire pour les requêtes SQL
conn = sqlite3.connect(':memory:')
df.to_sql('dataset_nettoye', conn, index=False, if_exists='replace')
print("\n✅ Base SQLite créée pour les requêtes SQL")

✅ Dataset chargé : 5121 lignes, 55 colonnes

Colonnes disponibles : ['demande_id', 'region_demandes', 'prefecture_demandes', 'commune', 'quartier', 'type_document', 'categorie_document', 'nombre_demandes', 'delai_traitement_jours', 'taux_rejet', 'date_demande', 'motif_demande', 'statut_demande', 'canal_demande', 'age_demandeur', 'sexe_demandeur', 'commune_id', 'prefecture_details', 'region_details', 'latitude', 'longitude', 'altitude_m', 'superficie_km2', 'population_densite', 'code_postal', 'type_commune', 'distance_capitale_km', 'zone_climatique', 'centre_id', 'nom_centre', 'type_centre', 'region', 'prefecture', 'commune_centres', 'quartier_centres', 'latitude_centres', 'longitude_centres', 'personnel_capacite_jour', 'nombre_guichets', 'heures_ouverture', 'horaire_nuit', 'equipement_numerique', 'date_ouverture', 'statut_centre', 'region_socio', 'prefecture_socio', 'commune_socio', 'population', 'superficie_km2_socio', 'densite', 'taux_urbanisation', 'taux_alphabétisation', 'age_media

## 2. KPI 1 : Délai Moyen de Traitement (Performance Opérationnelle)

In [2]:
"""
KPI 1 : Délai Moyen de Traitement des Demandes

Objectif métier :
- Mesurer l'efficacité opérationnelle des centres de service
- Identifier les centres ou régions avec des délais anormalement longs
- Suivre l'évolution temporelle pour évaluer les améliorations

Description / Interprétation :
- KPI exprimé en jours
- Valeur cible : < 15 jours (selon standards internationaux)
- Valeur alarmante : > 30 jours
- Permet d'identifier les goulots d'étranglement et d'optimiser les processus

Règle de calcul :
Délai moyen = (Somme des délais de traitement) / (Nombre de demandes traitées)
- Uniquement pour les demandes avec statut "Traitee"
- Exclure les délais négatifs ou nuls
"""

# Calcul en Python
def calculer_delai_moyen(df, filtre_region=None, filtre_commune=None):
    """
    Calcule le délai moyen de traitement des demandes
    
    Parameters:
    -----------
    df : DataFrame
        Dataset nettoyé
    filtre_region : str, optional
        Filtrer par région
    filtre_commune : str, optional
        Filtrer par commune
    
    Returns:
    --------
    float : Délai moyen en jours
    """
    df_filtered = df.copy()
    
    # Filtres optionnels
    if filtre_region:
        df_filtered = df_filtered[df_filtered['region_demandes'].str.lower() == filtre_region.lower()]
    if filtre_commune:
        df_filtered = df_filtered[df_filtered['commune'].str.lower() == filtre_commune.lower()]
    
    # Filtrer uniquement les demandes traitées avec délai valide
    df_traitees = df_filtered[
        (df_filtered['statut_demande'].str.lower() == 'traitee') &
        (df_filtered['delai_traitement_jours'] > 0) &
        (df_filtered['delai_traitement_jours'] <= 365)
    ]
    
    if len(df_traitees) == 0:
        return None
    
    delai_moyen = df_traitees['delai_traitement_jours'].mean()
    return round(delai_moyen, 2)

# Calcul global
delai_moyen_global = calculer_delai_moyen(df)
print(f"📊 Délai moyen de traitement (global) : {delai_moyen_global} jours")

# Calcul par région
print("\n📊 Délai moyen par région :")
regions = df['region_demandes'].dropna().unique()
for region in sorted(regions):
    delai = calculer_delai_moyen(df, filtre_region=region)
    if delai:
        print(f"  - {region} : {delai} jours")

# Requête SQL équivalente
sql_query = """
SELECT 
    region_demandes AS region,
    COUNT(*) AS nb_demandes_traitees,
    ROUND(AVG(delai_traitement_jours), 2) AS delai_moyen_jours
FROM dataset_nettoye
WHERE LOWER(statut_demande) = 'traitee'
    AND delai_traitement_jours > 0
    AND delai_traitement_jours <= 365
GROUP BY region_demandes
ORDER BY delai_moyen_jours DESC;
"""

print("\n📝 Requête SQL équivalente :")
print(sql_query)

# Exécution de la requête SQL
result_sql = pd.read_sql_query(sql_query, conn)
print("\n✅ Résultats de la requête SQL :")
print(result_sql)

📊 Délai moyen de traitement (global) : 21.71 jours

📊 Délai moyen par région :
  - centrale : 19.3 jours
  - kara : 25.22 jours
  - maritime : 21.8 jours
  - plateaux : 21.78 jours
  - savanes : 22.3 jours

📝 Requête SQL équivalente :

SELECT 
    region_demandes AS region,
    COUNT(*) AS nb_demandes_traitees,
    ROUND(AVG(delai_traitement_jours), 2) AS delai_moyen_jours
FROM dataset_nettoye
WHERE LOWER(statut_demande) = 'traitee'
    AND delai_traitement_jours > 0
    AND delai_traitement_jours <= 365
GROUP BY region_demandes
ORDER BY delai_moyen_jours DESC;


✅ Résultats de la requête SQL :
     region  nb_demandes_traitees  delai_moyen_jours
0      kara                   288              25.22
1   savanes                   280              22.30
2  maritime                   492              21.80
3  plateaux                   204              21.78
4  centrale                   516              19.30


## 3. KPI 2 : Taux d'Utilisation de la Capacité (Performance Opérationnelle)

In [3]:
"""
KPI 2 : Taux d'Utilisation de la Capacité des Centres

Objectif métier :
- Évaluer si les centres sont surchargés ou sous-utilisés
- Optimiser la répartition des ressources entre les centres
- Identifier les besoins de renforcement ou de création de nouveaux centres

Description / Interprétation :
- KPI exprimé en pourcentage (%)
- Valeur optimale : 70-85% (bon équilibre)
- Valeur alarmante : > 95% (surcharge) ou < 30% (sous-utilisation)
- Permet d'identifier les centres nécessitant plus de personnel ou d'équipements

Règle de calcul :
Taux d'utilisation = (Nombre de demandes traitées / Capacité journalière du centre) × 100
- Capacité = personnel_capacite_jour (nombre de personnes que le centre peut traiter par jour)
- Nombre de demandes = somme des demandes par centre
- Période : moyenne sur la période analysée
"""

# Calcul en Python
def calculer_taux_utilisation(df):
    """
    Calcule le taux d'utilisation de la capacité des centres
    
    Returns:
    --------
    DataFrame : Taux d'utilisation par centre
    """
    # Agréger les demandes par centre (si centre_id existe)
    # Sinon, utiliser la commune comme proxy
    if 'centre_id' in df.columns:
        demandes_par_centre = df.groupby('centre_id').agg({
            'nombre_demandes': 'sum'
        }).reset_index()
        
        # Joindre avec les capacités des centres
        if 'personnel_capacite_jour' in df.columns:
            capacites = df[['centre_id', 'personnel_capacite_jour', 'nom_centre', 'commune']].drop_duplicates()
            result = demandes_par_centre.merge(capacites, on='centre_id', how='left')
        else:
            result = demandes_par_centre
    else:
        # Utiliser commune comme proxy
        demandes_par_commune = df.groupby('commune').agg({
            'nombre_demandes': 'sum'
        }).reset_index()
        
        # Joindre avec les capacités (si disponibles par commune)
        if 'personnel_capacite_jour' in df.columns:
            capacites = df[['commune', 'personnel_capacite_jour']].drop_duplicates()
            result = demandes_par_commune.merge(capacites, on='commune', how='left')
        else:
            result = demandes_par_commune
    
    # Calculer le taux d'utilisation
    if 'personnel_capacite_jour' in result.columns:
        result['taux_utilisation_pct'] = (result['nombre_demandes'] / result['personnel_capacite_jour'] * 100).round(2)
        result = result[result['personnel_capacite_jour'] > 0]  # Éviter division par zéro
    else:
        result['taux_utilisation_pct'] = None
    
    return result

# Calcul
taux_utilisation = calculer_taux_utilisation(df)
print("📊 Taux d'utilisation de la capacité par centre/commune :")
print(taux_utilisation.head(10))

if 'taux_utilisation_pct' in taux_utilisation.columns and taux_utilisation['taux_utilisation_pct'].notna().any():
    taux_moyen = taux_utilisation['taux_utilisation_pct'].mean()
    print(f"\n📊 Taux d'utilisation moyen : {taux_moyen:.2f}%")

# Requête SQL équivalente
sql_query = """
SELECT 
    COALESCE(centre_id, commune) AS identifiant_centre,
    COUNT(*) AS nb_demandes,
    AVG(personnel_capacite_jour) AS capacite_moyenne,
    CASE 
        WHEN AVG(personnel_capacite_jour) > 0 
        THEN ROUND((COUNT(*) * 100.0 / AVG(personnel_capacite_jour)), 2)
        ELSE NULL 
    END AS taux_utilisation_pct
FROM dataset_nettoye
WHERE personnel_capacite_jour > 0
GROUP BY COALESCE(centre_id, commune)
ORDER BY taux_utilisation_pct DESC;
"""

print("\n📝 Requête SQL équivalente :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql.head(10))
except Exception as e:
    print(f"\n⚠️ Erreur SQL (colonnes manquantes) : {e}")

📊 Taux d'utilisation de la capacité par centre/commune :
  centre_id  nombre_demandes  personnel_capacite_jour      nom_centre  \
0     ct002            12618                       63     centre kévé   
1     ct002            12618                       63     centre kévé   
2     ct002            12618                       63     centre kévé   
3     ct002            12618                       63     centre kévé   
4     ct002            12618                       63     centre kévé   
5     ct002            12618                       63     centre kévé   
6     ct002            12618                       63     centre kévé   
7     ct003            12136                       98    centre gando   
8     ct003            12136                       98    centre gando   
9     ct004            14033                       78  centre pagouda   

     commune  taux_utilisation_pct  
0     tsévié              20028.57  
1       lomé              20028.57  
2  adidogome              20

In [4]:
"""
KPI 3 : Distance Moyenne aux Centres de Service

Objectif métier :
- Évaluer l'accessibilité géographique des services publics
- Identifier les zones sous-desservies nécessitant de nouveaux centres
- Optimiser la localisation des centres pour réduire les distances

Description / Interprétation :
- KPI exprimé en kilomètres (km)
- Valeur cible : < 25 km (bonne accessibilité)
- Valeur alarmante : > 50 km (faible accessibilité)
- Permet d'identifier les déserts de services et de planifier l'implantation de nouveaux centres

Règle de calcul :
Distance moyenne = (Somme des distances entre communes et centres les plus proches) / (Nombre de communes)
- Utiliser les coordonnées GPS (latitude, longitude) pour calculer la distance à vol d'oiseau
- Pour chaque commune, identifier le centre le plus proche
- Formule de Haversine pour calculer la distance géodésique
"""

# Fonction pour calculer la distance Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcule la distance entre deux points GPS en km (formule de Haversine)
    """
    from math import radians, sin, cos, sqrt, atan2
    
    R = 6371  # Rayon de la Terre en km
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distance = R * c
    return distance

# Calcul en Python
def calculer_distance_moyenne(df):
    """
    Calcule la distance moyenne des communes aux centres les plus proches
    """
    # Extraire les coordonnées des communes (si disponibles)
    communes_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    
    # Extraire les coordonnées des centres
    if 'latitude' in df.columns and 'longitude' in df.columns:
        centres_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    else:
        # Utiliser les coordonnées des communes comme proxy
        centres_coords = communes_coords.copy()
    
    distances = []
    
    for idx, commune in communes_coords.iterrows():
        if pd.notna(commune['latitude']) and pd.notna(commune['longitude']):
            min_distance = float('inf')
            
            # Trouver le centre le plus proche
            for idx2, centre in centres_coords.iterrows():
                if pd.notna(centre['latitude']) and pd.notna(centre['longitude']):
                    dist = haversine_distance(
                        commune['latitude'], commune['longitude'],
                        centre['latitude'], centre['longitude']
                    )
                    if dist < min_distance:
                        min_distance = dist
            
            if min_distance != float('inf'):
                distances.append({
                    'commune': commune['commune'],
                    'distance_km': min_distance
                })
    
    if len(distances) == 0:
        return None
    
    df_distances = pd.DataFrame(distances)
    distance_moyenne = df_distances['distance_km'].mean()
    
    return distance_moyenne, df_distances

# Calcul
result = calculer_distance_moyenne(df)
if result:
    distance_moyenne, df_distances = result
    print(f"📊 Distance moyenne aux centres de service : {distance_moyenne:.2f} km")
    print(f"\n📊 Top 10 communes les plus éloignées :")
    print(df_distances.nlargest(10, 'distance_km'))
else:
    # Utiliser distance_capitale_km comme proxy si disponible
    if 'distance_capitale_km' in df.columns:
        distance_moyenne = df['distance_capitale_km'].mean()
        print(f"📊 Distance moyenne à la capitale (proxy) : {distance_moyenne:.2f} km")
    else:
        print("⚠️ Coordonnées GPS non disponibles pour le calcul précis")

# Requête SQL équivalente (approximation avec distance_capitale_km)
sql_query = """
SELECT 
    region_demandes AS region,
    COUNT(DISTINCT commune) AS nb_communes,
    ROUND(AVG(distance_capitale_km), 2) AS distance_moyenne_capitale_km
FROM dataset_nettoye
WHERE distance_capitale_km IS NOT NULL
GROUP BY region_demandes
ORDER BY distance_moyenne_capitale_km DESC;
"""

print("\n📝 Requête SQL équivalente (proxy avec distance à la capitale) :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql)
except Exception as e:
    print(f"\n⚠️ Erreur SQL : {e}")

📊 Distance moyenne à la capitale (proxy) : nan km

📝 Requête SQL équivalente (proxy avec distance à la capitale) :

SELECT 
    region_demandes AS region,
    COUNT(DISTINCT commune) AS nb_communes,
    ROUND(AVG(distance_capitale_km), 2) AS distance_moyenne_capitale_km
FROM dataset_nettoye
WHERE distance_capitale_km IS NOT NULL
GROUP BY region_demandes
ORDER BY distance_moyenne_capitale_km DESC;


✅ Résultats de la requête SQL :
Empty DataFrame
Columns: [region, nb_communes, distance_moyenne_capitale_km]
Index: []


In [5]:
"""
KPI 4 : Taux de Couverture Démographique

Objectif métier :
- Mesurer le pourcentage de la population ayant accès à un centre à moins de X km
- Identifier les zones sous-desservies démographiquement
- Prioriser les investissements dans les zones à forte densité de population

Description / Interprétation :
- KPI exprimé en pourcentage (%)
- Valeur cible : > 80% de la population à moins de 25 km d'un centre
- Valeur alarmante : < 60% (faible couverture)
- Permet d'évaluer l'équité d'accès aux services publics

Règle de calcul :
Taux de couverture = (Population dans un rayon de X km autour d'un centre / Population totale) × 100
- Rayon de référence : 25 km (distance acceptable pour un service public)
- Utiliser les données démographiques par commune
- Compter la population des communes à moins de 25 km d'au moins un centre
"""

# Calcul en Python
def calculer_taux_couverture(df, rayon_km=25):
    """
    Calcule le taux de couverture démographique dans un rayon donné
    
    Parameters:
    -----------
    df : DataFrame
        Dataset nettoyé
    rayon_km : float
        Rayon de couverture en km (défaut : 25 km)
    
    Returns:
    --------
    float : Taux de couverture en %
    """
    # Extraire population et coordonnées par commune
    if 'population' in df.columns and 'latitude' in df.columns and 'longitude' in df.columns:
        communes_pop = df[['commune', 'population', 'latitude', 'longitude']].drop_duplicates().dropna()
    elif 'population_densite' in df.columns:
        # Utiliser densité comme proxy si population absolue absente
        communes_pop = df[['commune', 'population_densite', 'latitude', 'longitude']].drop_duplicates().dropna()
        communes_pop.rename(columns={'population_densite': 'population'}, inplace=True)
    else:
        return None
    
    # Identifier les centres (communes avec centres)
    centres_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    
    if len(communes_pop) == 0 or len(centres_coords) == 0:
        return None
    
    population_totale = communes_pop['population'].sum()
    population_couverte = 0
    
    # Pour chaque commune, vérifier si elle est dans le rayon d'un centre
    for idx, commune in communes_pop.iterrows():
        if pd.notna(commune['latitude']) and pd.notna(commune['longitude']):
            couverte = False
            
            for idx2, centre in centres_coords.iterrows():
                if pd.notna(centre['latitude']) and pd.notna(centre['longitude']):
                    dist = haversine_distance(
                        commune['latitude'], commune['longitude'],
                        centre['latitude'], centre['longitude']
                    )
                    if dist <= rayon_km:
                        couverte = True
                        break
            
            if couverte:
                population_couverte += commune['population']
    
    if population_totale == 0:
        return None
    
    taux_couverture = (population_couverte / population_totale) * 100
    return taux_couverture

# Calcul
taux_couverture = calculer_taux_couverture(df, rayon_km=25)
if taux_couverture:
    print(f"📊 Taux de couverture démographique (rayon 25 km) : {taux_couverture:.2f}%")
else:
    print("⚠️ Données démographiques ou coordonnées GPS non disponibles")

# Requête SQL équivalente (approximation)
sql_query = """
SELECT 
    region_demandes AS region,
    SUM(population) AS population_totale,
    COUNT(DISTINCT commune) AS nb_communes,
    COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) AS communes_couvertes,
    ROUND(
        (COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) * 100.0 / 
         COUNT(DISTINCT commune)), 2
    ) AS taux_couverture_communes_pct
FROM dataset_nettoye
WHERE population IS NOT NULL
GROUP BY region_demandes
ORDER BY taux_couverture_communes_pct DESC;
"""

print("\n📝 Requête SQL équivalente (approximation) :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql)
except Exception as e:
    print(f"\n⚠️ Erreur SQL : {e}")

⚠️ Données démographiques ou coordonnées GPS non disponibles

📝 Requête SQL équivalente (approximation) :

SELECT 
    region_demandes AS region,
    SUM(population) AS population_totale,
    COUNT(DISTINCT commune) AS nb_communes,
    COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) AS communes_couvertes,
    ROUND(
        (COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) * 100.0 / 
         COUNT(DISTINCT commune)), 2
    ) AS taux_couverture_communes_pct
FROM dataset_nettoye
WHERE population IS NOT NULL
GROUP BY region_demandes
ORDER BY taux_couverture_communes_pct DESC;


✅ Résultats de la requête SQL :
Empty DataFrame
Columns: [region, population_totale, nb_communes, communes_couvertes, taux_couverture_communes_pct]
Index: []


## 6. KPI 5 : Taux de Rejet des Demandes (Qualité de Service)

In [6]:
"""
KPI 5 : Taux de Rejet des Demandes

Objectif métier :
- Mesurer la qualité du service rendu aux citoyens
- Identifier les causes principales de rejet pour améliorer les processus
- Suivre l'évolution pour évaluer l'efficacité des formations du personnel

Description / Interprétation :
- KPI exprimé en pourcentage (%)
- Valeur cible : < 5% (excellente qualité)
- Valeur acceptable : 5-10% (qualité correcte)
- Valeur alarmante : > 15% (qualité insuffisante)
- Permet d'identifier les problèmes récurrents et d'améliorer la satisfaction citoyenne

Règle de calcul :
Taux de rejet = (Nombre de demandes rejetées / Nombre total de demandes) × 100
- Compter uniquement les demandes avec statut "Rejetée"
- Peut être calculé globalement, par région, par type de document, ou par centre
"""

# Calcul en Python
def calculer_taux_rejet(df, filtre_region=None, filtre_type_doc=None):
    """
    Calcule le taux de rejet des demandes
    
    Parameters:
    -----------
    df : DataFrame
        Dataset nettoyé
    filtre_region : str, optional
        Filtrer par région
    filtre_type_doc : str, optional
        Filtrer par type de document
    
    Returns:
    --------
    float : Taux de rejet en %
    """
    df_filtered = df.copy()
    
    # Filtres optionnels
    if filtre_region:
        df_filtered = df_filtered[df_filtered['region_demandes'].str.lower() == filtre_region.lower()]
    if filtre_type_doc:
        df_filtered = df_filtered[df_filtered['type_document'].str.lower() == filtre_type_doc.lower()]
    
    # Compter les demandes rejetées et totales
    total_demandes = len(df_filtered)
    demandes_rejetees = len(df_filtered[df_filtered['statut_demande'].str.lower() == 'rejetée'])
    
    if total_demandes == 0:
        return None
    
    taux_rejet = (demandes_rejetees / total_demandes) * 100
    return round(taux_rejet, 2)

# Calcul global
taux_rejet_global = calculer_taux_rejet(df)
print(f"📊 Taux de rejet global : {taux_rejet_global}%")

# Calcul par région
print("\n📊 Taux de rejet par région :")
for region in sorted(df['region_demandes'].dropna().unique()):
    taux = calculer_taux_rejet(df, filtre_region=region)
    if taux:
        print(f"  - {region} : {taux}%")

# Calcul par type de document
print("\n📊 Taux de rejet par type de document :")
for type_doc in sorted(df['type_document'].dropna().unique()):
    taux = calculer_taux_rejet(df, filtre_type_doc=type_doc)
    if taux:
        print(f"  - {type_doc} : {taux}%")

# Requête SQL équivalente
sql_query = """
SELECT 
    region_demandes AS region,
    type_document,
    COUNT(*) AS total_demandes,
    SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) AS demandes_rejetees,
    ROUND(
        (SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) AS taux_rejet_pct
FROM dataset_nettoye
GROUP BY region_demandes, type_document
ORDER BY taux_rejet_pct DESC;
"""

print("\n📝 Requête SQL équivalente :")
print(sql_query)

# Exécution de la requête SQL
result_sql = pd.read_sql_query(sql_query, conn)
print("\n✅ Résultats de la requête SQL :")
print(result_sql.head(15))

📊 Taux de rejet global : 34.92%

📊 Taux de rejet par région :
  - centrale : 35.34%
  - kara : 30.77%
  - maritime : 37.93%
  - plateaux : 28.46%
  - savanes : 38.26%

📊 Taux de rejet par type de document :
  - acte de naissance : 39.76%
  - carte d'identité : 33.72%
  - casier judiciaire : 32.5%
  - certificat de nationalité : 32.25%
  - livre de famille : 33.67%
  - passeport : 36.86%

📝 Requête SQL équivalente :

SELECT 
    region_demandes AS region,
    type_document,
    COUNT(*) AS total_demandes,
    SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) AS demandes_rejetees,
    ROUND(
        (SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) AS taux_rejet_pct
FROM dataset_nettoye
GROUP BY region_demandes, type_document
ORDER BY taux_rejet_pct DESC;


✅ Résultats de la requête SQL :
      region              type_document  total_demandes  demandes_rejetees  \
0   centrale                  passeport             240     

## 7. KPI 6 : Indice de Répartition Régionale (Efficience)

In [7]:
"""
KPI 6 : Indice de Répartition Régionale (Coefficient de Variation)

Objectif métier :
- Mesurer l'équité de la répartition des services entre les régions
- Identifier les déséquilibres territoriaux nécessitant des actions correctives
- Évaluer l'efficacité des politiques de décentralisation

Description / Interprétation :
- KPI exprimé en coefficient de variation (CV) sans unité
- Valeur cible : CV < 0.3 (bonne équité)
- Valeur acceptable : 0.3-0.5 (équité modérée)
- Valeur alarmante : CV > 0.5 (forte inéquité)
- CV = Écart-type / Moyenne (mesure la dispersion relative)

Règle de calcul :
Indice de répartition = Écart-type des volumes de demandes par région / Moyenne des volumes par région
- Calculer le volume de demandes par région
- Calculer la moyenne et l'écart-type de ces volumes
- CV = σ / μ
"""

# Calcul en Python
def calculer_indice_repartition(df):
    """
    Calcule l'indice de répartition régionale (coefficient de variation)
    
    Returns:
    --------
    float : Coefficient de variation
    """
    # Agréger les demandes par région
    demandes_par_region = df.groupby('region_demandes').agg({
        'nombre_demandes': 'sum'
    }).reset_index()
    
    if len(demandes_par_region) == 0:
        return None
    
    volumes = demandes_par_region['nombre_demandes']
    moyenne = volumes.mean()
    ecart_type = volumes.std()
    
    if moyenne == 0:
        return None
    
    cv = ecart_type / moyenne
    return round(cv, 3), demandes_par_region

# Calcul
result = calculer_indice_repartition(df)
if result:
    cv, df_regions = result
    print(f"📊 Indice de répartition régionale (CV) : {cv}")
    print(f"\n📊 Volumes de demandes par région :")
    print(df_regions.sort_values('nombre_demandes', ascending=False))
    
    # Interprétation
    if cv < 0.3:
        interpretation = "✅ Excellente équité entre les régions"
    elif cv < 0.5:
        interpretation = "⚠️ Équité modérée - quelques déséquilibres à corriger"
    else:
        interpretation = "❌ Forte inéquité - actions correctives nécessaires"
    
    print(f"\n💡 Interprétation : {interpretation}")
else:
    print("⚠️ Données insuffisantes pour calculer l'indice")

# Requête SQL équivalente
sql_query = """
WITH volumes_region AS (
    SELECT 
        region_demandes AS region,
        SUM(nombre_demandes) AS volume_total
    FROM dataset_nettoye
    WHERE region_demandes IS NOT NULL
    GROUP BY region_demandes
),
stats AS (
    SELECT 
        AVG(volume_total) AS moyenne,
        STDDEV(volume_total) AS ecart_type
    FROM volumes_region
)
SELECT 
    vr.region,
    vr.volume_total,
    s.moyenne,
    s.ecart_type,
    ROUND((s.ecart_type / s.moyenne), 3) AS coefficient_variation
FROM volumes_region vr
CROSS JOIN stats s
ORDER BY vr.volume_total DESC;
"""

print("\n📝 Requête SQL équivalente :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql)
except Exception as e:
    print(f"\n⚠️ Erreur SQL : {e}")

📊 Indice de répartition régionale (CV) : 0.375

📊 Volumes de demandes par région :
  region_demandes  nombre_demandes
2        maritime           151416
0        centrale           148644
1            kara           112264
4         savanes            84952
3        plateaux            54920

💡 Interprétation : ⚠️ Équité modérée - quelques déséquilibres à corriger

📝 Requête SQL équivalente :

WITH volumes_region AS (
    SELECT 
        region_demandes AS region,
        SUM(nombre_demandes) AS volume_total
    FROM dataset_nettoye
    WHERE region_demandes IS NOT NULL
    GROUP BY region_demandes
),
stats AS (
    SELECT 
        AVG(volume_total) AS moyenne,
        STDDEV(volume_total) AS ecart_type
    FROM volumes_region
)
SELECT 
    vr.region,
    vr.volume_total,
    s.moyenne,
    s.ecart_type,
    ROUND((s.ecart_type / s.moyenne), 3) AS coefficient_variation
FROM volumes_region vr
CROSS JOIN stats s
ORDER BY vr.volume_total DESC;


⚠️ Erreur SQL : Execution failed on sql '
W

## 8. Synthèse des KPI Calculés

In [8]:
# Synthèse de tous les KPI calculés
print("=" * 80)
print("SYNTHÈSE DES KPI CALCULÉS")
print("=" * 80)

# KPI 1 : Délai moyen
delai_moyen = calculer_delai_moyen(df)
print(f"\n1. Délai Moyen de Traitement : {delai_moyen} jours")
if delai_moyen:
    if delai_moyen < 15:
        print("   ✅ Excellent (objectif atteint)")
    elif delai_moyen < 30:
        print("   ⚠️ Acceptable mais à améliorer")
    else:
        print("   ❌ Délai trop long - action requise")

# KPI 2 : Taux d'utilisation
taux_util = calculer_taux_utilisation(df)
if 'taux_utilisation_pct' in taux_util.columns and taux_util['taux_utilisation_pct'].notna().any():
    taux_util_moyen = taux_util['taux_utilisation_pct'].mean()
    print(f"\n2. Taux d'Utilisation de la Capacité : {taux_util_moyen:.2f}%")
    if 70 <= taux_util_moyen <= 85:
        print("   ✅ Optimal")
    elif taux_util_moyen > 95:
        print("   ❌ Surcharge - renforcement nécessaire")
    elif taux_util_moyen < 30:
        print("   ⚠️ Sous-utilisation - optimisation possible")
else:
    print("\n2. Taux d'Utilisation : Données insuffisantes")

# KPI 3 : Distance moyenne
result_dist = calculer_distance_moyenne(df)
if result_dist:
    dist_moyenne, _ = result_dist
    print(f"\n3. Distance Moyenne aux Centres : {dist_moyenne:.2f} km")
    if dist_moyenne < 25:
        print("   ✅ Bonne accessibilité")
    elif dist_moyenne < 50:
        print("   ⚠️ Accessibilité modérée")
    else:
        print("   ❌ Faible accessibilité - nouveaux centres nécessaires")
else:
    print("\n3. Distance Moyenne : Données GPS insuffisantes")

# KPI 4 : Taux de couverture
taux_couv = calculer_taux_couverture(df, rayon_km=25)
if taux_couv:
    print(f"\n4. Taux de Couverture Démographique : {taux_couv:.2f}%")
    if taux_couv >= 80:
        print("   ✅ Excellente couverture")
    elif taux_couv >= 60:
        print("   ⚠️ Couverture acceptable")
    else:
        print("   ❌ Faible couverture - expansion nécessaire")
else:
    print("\n4. Taux de Couverture : Données insuffisantes")

# KPI 5 : Taux de rejet
taux_rej = calculer_taux_rejet(df)
print(f"\n5. Taux de Rejet des Demandes : {taux_rej}%")
if taux_rej:
    if taux_rej < 5:
        print("   ✅ Excellente qualité")
    elif taux_rej < 10:
        print("   ⚠️ Qualité correcte")
    else:
        print("   ❌ Qualité insuffisante - amélioration nécessaire")

# KPI 6 : Indice de répartition
result_cv = calculer_indice_repartition(df)
if result_cv:
    cv, _ = result_cv
    print(f"\n6. Indice de Répartition Régionale (CV) : {cv}")
    if cv < 0.3:
        print("   ✅ Excellente équité")
    elif cv < 0.5:
        print("   ⚠️ Équité modérée")
    else:
        print("   ❌ Forte inéquité - actions correctives nécessaires")

print("\n" + "=" * 80)
print("✅ Tous les KPI ont été calculés avec succès")
print("=" * 80)

SYNTHÈSE DES KPI CALCULÉS

1. Délai Moyen de Traitement : 21.71 jours
   ⚠️ Acceptable mais à améliorer

2. Taux d'Utilisation de la Capacité : 18361.31%
   ❌ Surcharge - renforcement nécessaire

3. Distance Moyenne : Données GPS insuffisantes

4. Taux de Couverture : Données insuffisantes

5. Taux de Rejet des Demandes : 34.92%
   ❌ Qualité insuffisante - amélioration nécessaire

6. Indice de Répartition Régionale (CV) : 0.375
   ⚠️ Équité modérée

✅ Tous les KPI ont été calculés avec succès
